In [1]:
import requests, six
import pandas as pd

In [2]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [3]:
import lxml.html as lh

In [4]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'#Create a handle, page, to handle the contents of the website

# Inspect webpage with Inspect Element (Q) of Firefox browser
![Inspect Element (Q)](firefoxinspect.JPG)

In [5]:
page = requests.get(url)#Store the contents of the website under doc
doc = lh.fromstring(page.content)#Parse data that locate between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [6]:
#Check the length of the first 10 rows
[len(T) for T in tr_elements[:10]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [7]:
tr_elements = doc.xpath('//tr')#Create empty list
col=[]
i=0 # Fill the first row with the "headers" or name of columns.
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:'"%s"''%(i,name))
    col.append((name,[]))

1:Postal Code

2:Borough

3:Neighborhood



In [8]:
# Start fill data in the second row

for j in range(1,len(tr_elements)):
    
    # R is j'th row
    R=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(R) != 3:
    
     break    
    
    # i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in R.iterchildren():
        data=t.text_content() 
        
        #Check if row is empty
        if i>0:        
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        
        i+=1

In [9]:
# Inspect the content in each column
[len(C) for (title,C) in col]

[181, 181, 181]

In [10]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)

In [11]:
df.head(5)

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [12]:
df.tail(5)

,Postal Code\n,Borough\n,Neighborhood\n
176,M6Z\n,Not assigned\n,Not assigned\n
177,M7Z\n,Not assigned\n,Not assigned\n
178,M8Z\n,Etobicoke\n,"Mimico NW, The Queensway West, South of Bloor,..."
179,M9Z\n,Not assigned\n,Not assigned\n
180,\n,Canadian postal codes\n,\n


In [13]:
print(df.dtypes)

Postal Code\n     object
Borough\n         object
Neighborhood\n    object
dtype: object


In [14]:
# Rename the column name, drop all '\n', and delete the last row "Canadian Postal codes"
df.columns=["PostalCode", "Borough","Neighborhood"]
df = df.replace('\n','', regex=True)
df.drop(df.index[180], inplace = True)

In [15]:
df['Borough'].value_counts()


Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [16]:
# Delete all rows having " Not assigned" in the "Borough" column

df.drop(df[df['Borough'] == "Not assigned"].index, inplace = True) 

In [17]:
# Check if the "Not assigned" value in the Neighborhood column.
count = df[df['Neighborhood']=="Not assigned"]
print(count)

Empty DataFrame
Columns: [PostalCode, Borough, Neighborhood]
Index: []


In [18]:
df.reset_index(drop=True, inplace=True)
df.head(12)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [19]:
print(df.shape)

(103, 3)
